In [1]:
import requests
import json
import numpy as np 
import pandas as pd
import os
import pickle

In [2]:
cases_endpt = 'https://api.gdc.cancer.gov/cases'

In [3]:
files_endpt = "https://api.gdc.cancer.gov/files"

In [4]:
import requests
import json

fields = [
    "file_name",
    "cases.submitter_id",
    "cases.samples.sample_type",
    "cases.disease_type",
    "cases.project.project_id"
    ]

fields = ",".join(fields)

files_endpt = "https://api.gdc.cancer.gov/files"

# This set of filters is nested under an 'and' operator.
filters = {
    "op": "and",
    "content":[
        {
        "op": "in",
        "content":{
            "field": "cases.project.primary_site",
            "value": ["Lung"]
            }
        },
        {
        "op": "in",
        "content":{
            "field": "files.experimental_strategy",
            "value": ["RNA-Seq"]
            }
        },
        {
        "op": "in",
        "content":{
            "field": "files.data_format",
            "value": ["BAM"]
            }
        }
    ]
}

# A POST is used, so the filter parameters can be passed directly as a Dict object.
params = {
    "filters": filters,
    "fields": fields,
    "format": "TSV",
    "size": "5"
    }

# The parameters are passed to 'json' rather than 'params' in this case
response = requests.post(files_endpt, headers = {"Content-Type": "application/json"}, json = params)

print(response.content.decode("utf-8"))

cases.0.disease_type	cases.0.project.project_id	cases.0.samples.0.sample_type	cases.0.submitter_id	file_name	id
Squamous Cell Neoplasms	TCGA-LUSC	Primary Tumor	TCGA-85-6561	8a52e74d-e830-4c13-81e1-caaf032b4549_gdc_realn_rehead.bam	73e7a49e-87fd-4809-a076-1394a1a619a2
Squamous Cell Neoplasms	TCGA-LUSC	Primary Tumor	TCGA-77-8144	fc131a10-fa8f-45da-8732-642cc45c91e6_gdc_realn_rehead.bam	a27ca6ec-7c8b-45cf-9daf-0bff9fdb66f0
Squamous Cell Neoplasms	TCGA-LUSC	Primary Tumor	TCGA-18-3411	ddcab400-d9a0-40da-907f-adc5805c6bbb_gdc_realn_rehead.bam	e0979f1e-e816-4322-b1e1-cde921032091
Squamous Cell Neoplasms	TCGA-LUSC	Solid Tissue Normal	TCGA-56-7582	8ef3c107-48d0-4203-820f-5b70394fe261_gdc_realn_rehead.bam	ddeeaa88-e030-4d0b-9a46-531323461bb9
Squamous Cell Neoplasms	TCGA-LUSC	Primary Tumor	TCGA-43-7658	9560e4b7-5f34-4139-8280-faf4e5b97a22_gdc_realn_rehead.bam	03bb0e2c-7a1c-4195-92a2-7385b19f87f9



In [5]:
#writing my own actual request - 
# goal - get RNA-seq for primary tumor, caseid, primary site, drug used, progression free days or equivalent

#first table just need to be caseid, primary site for files that have rna seq and drug used 

fields_table0 = [
    "case_id",
    "primary_site",
    "project_id",
    "diagnoses.days_to_recurrence",
    "diagnoses.last_known_disease_status",
    "diagnoses.progression_or_recurrence",
    "diagnoses.treatments.therapeutic_agents",
    "diagnoses.treatments.treatment_or_therapy",
    "diagnoses.vital_status"
    ]

fields_table1 = ["case_id","primary_site","project.name", "project.project_id"] 
#project_id is the same as case_id, program.program_id isnt showing up at all

array_reshape_value = len(fields_table1)


In [6]:
fields_table1 = ",".join(fields_table1)

In [7]:
filters_table1 = {
    "op":"and",
    "content":[{"op":"in", 
                "content":{
        "field":"primary_site",
        "value": ["Kidney"]}}, 
                {"op":"in",
                 "content":{
                     "field":"files.experimental_strategy", 
                     "value":["RNA-Seq"]}}]}

In [8]:
params_table1 = {
    "filters": json.dumps(filters_table1),
    "fields": fields_table1,
    "format": "TSV",
    "size": "5000"
    }

In [9]:
response_table1 = requests.get(cases_endpt, params = params_table1)
#response_table1_1 = requests.post(cases_endpt, headers = {"Content-Type": "application/json"}, json = params)


In [10]:
clinical_data = response_table1.content.decode("utf-8")
clinical_data

'case_id\tid\tprimary_site\tproject.name\tproject.project_id\r\n5338d435-68fb-4f0d-a3e6-c843f703f75f\t5338d435-68fb-4f0d-a3e6-c843f703f75f\tKidney\tKidney Renal Clear Cell Carcinoma\tTCGA-KIRC\r\n22b6724c-a59f-4796-8166-992253e8caf1\t22b6724c-a59f-4796-8166-992253e8caf1\tKidney\tKidney Renal Clear Cell Carcinoma\tTCGA-KIRC\r\n5c73b91a-2bee-4388-854c-1c3a6ae732db\t5c73b91a-2bee-4388-854c-1c3a6ae732db\tKidney\tKidney Renal Clear Cell Carcinoma\tTCGA-KIRC\r\nd5213422-9447-4f62-b486-3303e3f991fa\td5213422-9447-4f62-b486-3303e3f991fa\tKidney\tKidney Renal Clear Cell Carcinoma\tTCGA-KIRC\r\na7239dcc-18a7-5c96-bb95-3b117e196e6f\ta7239dcc-18a7-5c96-bb95-3b117e196e6f\tKidney\tHigh-Risk Wilms Tumor\tTARGET-WT\r\n8d36b806-163d-5ad1-83f0-ccdc6e728e94\t8d36b806-163d-5ad1-83f0-ccdc6e728e94\tKidney\tHigh-Risk Wilms Tumor\tTARGET-WT\r\nd9f4709d-5d3c-5ae3-8c2d-07466f088e0d\td9f4709d-5d3c-5ae3-8c2d-07466f088e0d\tKidney\tHigh-Risk Wilms Tumor\tTARGET-WT\r\n4dcdd89c-b2a7-594c-95d7-f0e17b7ba440\t4dcdd89c-b

In [11]:
clinical_data_processed = clinical_data.replace("\t", ",")
clinical_data_processed = clinical_data_processed.replace("\r\n", ",")

In [12]:
cd = clinical_data_processed.split(",")

In [13]:
len(cd)

5996

In [14]:
cd_array = np.asarray(cd[0:-1]) # not sure how to correctly just deal with this one last element

In [15]:
cd_array = cd_array.reshape(-1, 5)

In [16]:
cd_array[0]

array(['case_id', 'id', 'primary_site', 'project.name',
       'project.project_id'], dtype='<U37')

In [17]:
cd_df = pd.DataFrame(cd_array[1:], columns=cd_array[0])
cd_df.rename(columns = {"project.name":"project_name", "project.project_id":"project_id"}, inplace = True)

In [18]:
cd_df #this is way more than were available through the online portal

,case_id,id,primary_site,project_name,project_id
0,5338d435-68fb-4f0d-a3e6-c843f703f75f,5338d435-68fb-4f0d-a3e6-c843f703f75f,Kidney,Kidney Renal Clear Cell Carcinoma,TCGA-KIRC
1,22b6724c-a59f-4796-8166-992253e8caf1,22b6724c-a59f-4796-8166-992253e8caf1,Kidney,Kidney Renal Clear Cell Carcinoma,TCGA-KIRC
2,5c73b91a-2bee-4388-854c-1c3a6ae732db,5c73b91a-2bee-4388-854c-1c3a6ae732db,Kidney,Kidney Renal Clear Cell Carcinoma,TCGA-KIRC
3,d5213422-9447-4f62-b486-3303e3f991fa,d5213422-9447-4f62-b486-3303e3f991fa,Kidney,Kidney Renal Clear Cell Carcinoma,TCGA-KIRC
4,a7239dcc-18a7-5c96-bb95-3b117e196e6f,a7239dcc-18a7-5c96-bb95-3b117e196e6f,Kidney,High-Risk Wilms Tumor,TARGET-WT
...,...,...,...,...,...
1193,28011111-4a01-4cdc-8d6b-7223fb2c501b,28011111-4a01-4cdc-8d6b-7223fb2c501b,Kidney,Kidney Renal Clear Cell Carcinoma,TCGA-KIRC
1194,659294b9-ded9-498a-bc00-c1d4d456aa4a,659294b9-ded9-498a-bc00-c1d4d456aa4a,Kidney,Kidney Renal Clear Cell Carcinoma,TCGA-KIRC
1195,4519a839-11ea-4628-b5a7-071833ad16de,4519a839-11ea-4628-b5a7-071833ad16de,Kidney,Kidney Renal Clear Cell Carcinoma,TCGA-KIRC
1196,d0f908c5-aafd-4b76-a2fb-39b7cc19bcf7,d0f908c5-aafd-4b76-a2fb-39b7cc19bcf7,Kidney,Kidney Renal Clear Cell Carcinoma,TCGA-KIRC


In [19]:
cd_df.project_id.value_counts()

TCGA-KIRC      530
TCGA-KIRP      289
TARGET-WT      125
CPTAC-3        110
TCGA-KICH       66
TARGET-RT       57
TARGET-CCSK     13
TARGET-NBL       6
TCGA-SARC        2
Name: project_id, dtype: int64

In [20]:
cd_df.primary_site.value_counts()

Kidney    1198
Name: primary_site, dtype: int64

In [21]:
cd_df[cd_df['project_id']=='TARGET-WT'] #clinical records for this are connected to case id, not uiud

,case_id,id,primary_site,project_name,project_id
4,a7239dcc-18a7-5c96-bb95-3b117e196e6f,a7239dcc-18a7-5c96-bb95-3b117e196e6f,Kidney,High-Risk Wilms Tumor,TARGET-WT
5,8d36b806-163d-5ad1-83f0-ccdc6e728e94,8d36b806-163d-5ad1-83f0-ccdc6e728e94,Kidney,High-Risk Wilms Tumor,TARGET-WT
6,d9f4709d-5d3c-5ae3-8c2d-07466f088e0d,d9f4709d-5d3c-5ae3-8c2d-07466f088e0d,Kidney,High-Risk Wilms Tumor,TARGET-WT
7,4dcdd89c-b2a7-594c-95d7-f0e17b7ba440,4dcdd89c-b2a7-594c-95d7-f0e17b7ba440,Kidney,High-Risk Wilms Tumor,TARGET-WT
8,6163ffa9-e0b9-5d6f-b650-0ce2df32b6c3,6163ffa9-e0b9-5d6f-b650-0ce2df32b6c3,Kidney,High-Risk Wilms Tumor,TARGET-WT
...,...,...,...,...,...
1015,37b63d05-d4fe-544e-b215-9751ab9d4e93,37b63d05-d4fe-544e-b215-9751ab9d4e93,Kidney,High-Risk Wilms Tumor,TARGET-WT
1016,214e8f3a-0229-583b-935d-a5c64b3aa58e,214e8f3a-0229-583b-935d-a5c64b3aa58e,Kidney,High-Risk Wilms Tumor,TARGET-WT
1017,febc157e-ad8d-572b-903c-99752948f367,febc157e-ad8d-572b-903c-99752948f367,Kidney,High-Risk Wilms Tumor,TARGET-WT
1018,b924440c-c470-5877-980a-6665b47d3329,b924440c-c470-5877-980a-6665b47d3329,Kidney,High-Risk Wilms Tumor,TARGET-WT


In [22]:
cd_df[cd_df['project_id']=='CPTAC-3'] #can't find clinical info here too

,case_id,id,primary_site,project_name,project_id
206,022c3490-811c-4f82-ad9b-8004a3df7e5c,022c3490-811c-4f82-ad9b-8004a3df7e5c,Kidney,,CPTAC-3
207,e46562b6-3f3e-4886-ade9-43a35cc05c79,e46562b6-3f3e-4886-ade9-43a35cc05c79,Kidney,,CPTAC-3
208,f53d22a7-314a-48c0-b8b1-9799149ccc1e,f53d22a7-314a-48c0-b8b1-9799149ccc1e,Kidney,,CPTAC-3
209,aa62b9ec-062b-41c9-abb4-c7217f259c06,aa62b9ec-062b-41c9-abb4-c7217f259c06,Kidney,,CPTAC-3
210,7e18715d-3d61-431b-94f5-4d20bbaa4fa1,7e18715d-3d61-431b-94f5-4d20bbaa4fa1,Kidney,,CPTAC-3
...,...,...,...,...,...
1186,ad688008-ae8d-4bfd-99ab-24998a2c248c,ad688008-ae8d-4bfd-99ab-24998a2c248c,Kidney,,CPTAC-3
1187,629dc6f4-9e1c-4265-9fd6-838ca8222792,629dc6f4-9e1c-4265-9fd6-838ca8222792,Kidney,,CPTAC-3
1188,31828179-fd00-4918-96b7-01b7c894e22d,31828179-fd00-4918-96b7-01b7c894e22d,Kidney,,CPTAC-3
1189,ceae0345-0b5a-475f-8133-5fffcb81f3d0,ceae0345-0b5a-475f-8133-5fffcb81f3d0,Kidney,,CPTAC-3


In [23]:
cd_df[cd_df['project_id']=='TCGA-KICH'] #all data available, but drugs only for a select few

,case_id,id,primary_site,project_name,project_id
117,03f3dd32-e1d1-485b-a968-3f55798f4d46,03f3dd32-e1d1-485b-a968-3f55798f4d46,Kidney,Kidney Chromophobe,TCGA-KICH
118,9aa02d46-294b-4e99-b9ee-56edcbfb5275,9aa02d46-294b-4e99-b9ee-56edcbfb5275,Kidney,Kidney Chromophobe,TCGA-KICH
119,0f3c4f8f-0766-46b0-81b4-18ca4f6ea20a,0f3c4f8f-0766-46b0-81b4-18ca4f6ea20a,Kidney,Kidney Chromophobe,TCGA-KICH
120,452e96b0-dd23-4f5a-bc05-625763b0ee12,452e96b0-dd23-4f5a-bc05-625763b0ee12,Kidney,Kidney Chromophobe,TCGA-KICH
121,9f8aa332-e625-46f7-b77b-1fcb3ca99ee2,9f8aa332-e625-46f7-b77b-1fcb3ca99ee2,Kidney,Kidney Chromophobe,TCGA-KICH
...,...,...,...,...,...
985,2a05f855-83cc-4276-befd-5d6d4ec18f82,2a05f855-83cc-4276-befd-5d6d4ec18f82,Kidney,Kidney Chromophobe,TCGA-KICH
986,d52c3cb6-921e-4f26-b49c-e6e4f11f9c06,d52c3cb6-921e-4f26-b49c-e6e4f11f9c06,Kidney,Kidney Chromophobe,TCGA-KICH
989,452135f2-6de6-4593-a091-ddf6344ee431,452135f2-6de6-4593-a091-ddf6344ee431,Kidney,Kidney Chromophobe,TCGA-KICH
1025,a5af1391-89d3-4f41-930e-7e8272afec98,a5af1391-89d3-4f41-930e-7e8272afec98,Kidney,Kidney Chromophobe,TCGA-KICH


In [24]:
cd_df[cd_df['project_id']=='TARGET-RT'] #same problem as target-wt - no drug info

,case_id,id,primary_site,project_name,project_id
24,26079d5f-b061-5a39-968c-199e017cb880,26079d5f-b061-5a39-968c-199e017cb880,Kidney,Rhabdoid Tumor,TARGET-RT
25,bb8a0b9d-dbc5-50e6-89e0-e129305fa2ad,bb8a0b9d-dbc5-50e6-89e0-e129305fa2ad,Kidney,Rhabdoid Tumor,TARGET-RT
26,122eba47-a308-5c80-a332-06b37f12cc3a,122eba47-a308-5c80-a332-06b37f12cc3a,Kidney,Rhabdoid Tumor,TARGET-RT
27,105d2431-78e7-5e34-b620-0b9d4fd9d1a3,105d2431-78e7-5e34-b620-0b9d4fd9d1a3,Kidney,Rhabdoid Tumor,TARGET-RT
28,ef42d20f-5f3e-58c6-9d52-0a2aa48a162c,ef42d20f-5f3e-58c6-9d52-0a2aa48a162c,Kidney,Rhabdoid Tumor,TARGET-RT
29,9e15d908-12c2-5a1b-b1c4-c328242d474a,9e15d908-12c2-5a1b-b1c4-c328242d474a,Kidney,Rhabdoid Tumor,TARGET-RT
30,b34991f5-fa49-5b35-8e1b-95794d23ca12,b34991f5-fa49-5b35-8e1b-95794d23ca12,Kidney,Rhabdoid Tumor,TARGET-RT
31,2e3c665d-8b99-5034-9416-de02275682b2,2e3c665d-8b99-5034-9416-de02275682b2,Kidney,Rhabdoid Tumor,TARGET-RT
32,32951728-1cf0-54ce-9a96-ebc5e0aeec3d,32951728-1cf0-54ce-9a96-ebc5e0aeec3d,Kidney,Rhabdoid Tumor,TARGET-RT
33,3e859917-bef2-55d5-afc0-dffa8e6cd12e,3e859917-bef2-55d5-afc0-dffa8e6cd12e,Kidney,Rhabdoid Tumor,TARGET-RT


In [25]:
cd_df[cd_df['project_id']=='TARGET-CCSK'] #same as previous target project issues

,case_id,id,primary_site,project_name,project_id
102,702290ef-d941-5f40-aaa5-705477c9236d,702290ef-d941-5f40-aaa5-705477c9236d,Kidney,Clear Cell Sarcoma of the Kidney,TARGET-CCSK
103,81177083-91dc-56e7-bf0e-474031411414,81177083-91dc-56e7-bf0e-474031411414,Kidney,Clear Cell Sarcoma of the Kidney,TARGET-CCSK
104,289ddd3b-f74f-5669-a96f-6a9368952618,289ddd3b-f74f-5669-a96f-6a9368952618,Kidney,Clear Cell Sarcoma of the Kidney,TARGET-CCSK
105,12a3eea5-ba59-5f2e-ba2a-aaedbf2246f2,12a3eea5-ba59-5f2e-ba2a-aaedbf2246f2,Kidney,Clear Cell Sarcoma of the Kidney,TARGET-CCSK
106,754e2719-9d5d-53d9-a37f-7e8dd7301694,754e2719-9d5d-53d9-a37f-7e8dd7301694,Kidney,Clear Cell Sarcoma of the Kidney,TARGET-CCSK
129,b7accd91-4c42-5522-b8bd-8350e9f7a202,b7accd91-4c42-5522-b8bd-8350e9f7a202,Kidney,Clear Cell Sarcoma of the Kidney,TARGET-CCSK
259,0008bdfb-24a3-50fa-b112-89966d6ca423,0008bdfb-24a3-50fa-b112-89966d6ca423,Kidney,Clear Cell Sarcoma of the Kidney,TARGET-CCSK
260,d9bbb9dd-5a89-58c1-9ad6-578f1ed85962,d9bbb9dd-5a89-58c1-9ad6-578f1ed85962,Kidney,Clear Cell Sarcoma of the Kidney,TARGET-CCSK
261,f812897d-0449-58a6-b08c-2a6e539bac1a,f812897d-0449-58a6-b08c-2a6e539bac1a,Kidney,Clear Cell Sarcoma of the Kidney,TARGET-CCSK
571,47da0371-6760-56af-9c3a-8e42ec1a83ff,47da0371-6760-56af-9c3a-8e42ec1a83ff,Kidney,Clear Cell Sarcoma of the Kidney,TARGET-CCSK


In [26]:
cd_df[cd_df['project_id']=='TCGA-SARC'] #clinical data pulled

,case_id,id,primary_site,project_name,project_id
388,109df703-2865-4b4c-8e4b-badff8eb3f22,109df703-2865-4b4c-8e4b-badff8eb3f22,Kidney,Sarcoma,TCGA-SARC
1111,479d7a5a-63dc-412b-9075-920525334fe7,479d7a5a-63dc-412b-9075-920525334fe7,Kidney,Sarcoma,TCGA-SARC


In [4]:
root_dir = '/Users/dinakats/Desktop/SPICED/final_proj_git_renew/Genetics-to-Therapuetics/Data/Kidney/clinical_data'

In [5]:
files_list  = os.listdir(root_dir)
files_list

['nationwidechildrens.org_clinical_drug_kirc.txt',
 'nationwidechildrens.org_clinical_follow_up_v1.0_kirc.txt',
 'TARGET_WT_ClinicalData_Discovery_20170525.xlsx',
 '.DS_Store',
 'nationwidechildrens.org_clinical_follow_up_v1.0_kirp.txt',
 'nationwidechildrens.org_clinical_drug_kirp.txt',
 'nationwidechildrens.org_clinical_patient_sarc.txt',
 'nationwidechildrens.org_clinical_drug_kich.txt',
 'TARGET_WT_ClinicalData_Discovery_and_Validation_Percent_Tumor_Nuclei_and_Necrosis_Supplement_20170525.xlsx',
 'nationwidechildrens.org_clinical_follow_up_v4.0_nte_sarc.txt',
 'nationwidechildrens.org_clinical_follow_up_v4.4_kich.txt',
 'TARGET_WT_CDE_20170525.xlsx',
 'nationwidechildrens.org_clinical_patient_kich.txt',
 'nationwidechildrens.org_clinical_patient_kirp.txt',
 'nationwidechildrens.org_clinical_drug_sarc.txt',
 'nationwidechildrens.org_clinical_follow_up_v4.4_nte_kich.txt',
 'nationwidechildrens.org_clinical_patient_kirc.txt',
 'nationwidechildrens.org_clinical_follow_up_v4.0_sarc.txt'

In [29]:
# import clinical data sheet for each study 
drug_kirc = pd.read_csv(root_dir+'/'+'nationwidechildrens.org_clinical_drug_kirc.txt', sep = '\t', skiprows= 1)
followup_kirc = pd.read_csv(root_dir+'/'+files_list[1], sep = '\t', skiprows= 1)

In [30]:
followup_kirc

,bcr_patient_uuid,bcr_patient_barcode,bcr_followup_barcode,bcr_followup_uuid,form_completion_date,followup_case_report_form_submission_reason,lost_follow_up,radiation_therapy,postoperative_rx_tx,primary_therapy_outcome_success,...,performance_status_scale_timing,new_tumor_event_after_initial_treatment,days_to_new_tumor_event_after_initial_treatment,additional_surgery_locoregional_procedure,days_to_additional_surgery_locoregional_procedure,additional_surgery_metastatic_procedure,days_to_additional_surgery_metastatic_procedure,additional_radiation_therapy,additional_pharmaceutical_therapy,followup_treatment_success
0,CDE_ID:,CDE_ID:2003301,CDE_ID:,CDE_ID:,CDE_ID:,CDE_ID:3233305,CDE_ID:61333,CDE_ID:2005312,CDE_ID:3397567,CDE_ID:2786727,...,CDE_ID:2792763,CDE_ID:3121376,CDE_ID:3392464,CDE_ID:3008755,CDE_ID:3408572,CDE_ID:3008757,CDE_ID:3408682,CDE_ID:3427615,CDE_ID:3427616,CDE_ID:3104050
1,2B1DEA0A-6D55-4FDD-9C1C-0D9FBE03BD78,TCGA-3Z-A93Z,TCGA-3Z-A93Z-F67780,34AD0BAB-8A49-40F9-8C37-7BFE14390FE9,2014-11-11,Scheduled Follow-up Submission,NO,NO,NO,Complete Remission/Response,...,[Unknown],NO,[Not Applicable],[Not Available],[Not Applicable],[Not Available],[Not Applicable],[Not Available],[Not Available],Complete Remission/Response
2,D3B47E53-6F40-4FC8-B5A4-CBE548A770A9,TCGA-6D-AA2E,TCGA-6D-AA2E-F62639,7D287EC0-3348-4943-A62D-4CDF3A5BA81E,2014-7-22,Scheduled Follow-up Submission,NO,NO,NO,[Unknown],...,[Unknown],NO,[Not Applicable],[Not Available],[Not Applicable],[Not Available],[Not Applicable],[Not Available],[Not Available],Partial Remission/Response
3,3cbca837-f5a7-4a87-8f02-c59eac232d5a,TCGA-A3-3308,TCGA-A3-3308-F466,33f87611-107f-445a-b8f1-5982a8fc352a,2010-4-12,[Not Available],[Not Available],[Not Available],[Not Available],[Not Available],...,[Not Available],[Not Available],[Not Available],[Not Available],[Not Available],[Not Available],[Not Available],[Not Available],[Not Available],[Not Available]
4,0c139772-e303-45d1-b0c7-438fa1db105e,TCGA-A3-3311,TCGA-A3-3311-F468,a275b2f4-0d9a-4995-b28a-69bfd4778b53,2010-5-11,[Not Available],[Not Available],[Not Available],[Not Available],[Not Available],...,[Not Available],[Not Available],[Not Available],[Not Available],[Not Available],[Not Available],[Not Available],[Not Available],[Not Available],[Not Available]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
699,FE1F95A3-A6C2-4A39-8586-F8A70E975DD4,TCGA-MM-A563,TCGA-MM-A563-F57283,E3EF05E9-D6A1-4ACB-A753-AAC265999E16,2014-3-7,Scheduled Follow-up Submission,NO,NO,NO,[Unknown],...,[Not Evaluated],NO,[Not Applicable],[Not Available],[Not Applicable],[Not Available],[Not Applicable],[Not Available],[Not Available],Stable Disease
700,620BAEFA-DB8F-4AF9-9101-4B9B252FA73E,TCGA-MM-A564,TCGA-MM-A564-F56670,73387097-68B0-4648-BBC2-B3F97410A411,2014-2-17,Scheduled Follow-up Submission,NO,NO,NO,Complete Remission/Response,...,[Not Available],NO,[Not Applicable],[Not Available],[Not Applicable],[Not Available],[Not Applicable],[Not Available],[Not Available],[Not Available]
701,AA2B6825-3352-4D47-A637-48452F4A958B,TCGA-MM-A84U,TCGA-MM-A84U-F62896,CA22ACFE-9BB5-4002-9B7B-15BD157D4E42,2014-7-28,Scheduled Follow-up Submission,NO,NO,NO,Complete Remission/Response,...,[Not Evaluated],NO,[Not Applicable],[Not Available],[Not Applicable],[Not Available],[Not Applicable],[Not Available],[Not Available],Complete Remission/Response
702,B123E7C1-2472-4427-BFEB-79666F971E66,TCGA-MW-A4EC,TCGA-MW-A4EC-F70586,9BA3D0DB-F7AB-462E-9CC5-569033D21796,2015-2-27,Scheduled Follow-up Submission,NO,NO,NO,[Not Available],...,[Not Evaluated],NO,[Not Applicable],[Not Available],[Not Applicable],[Not Available],[Not Applicable],[Not Available],[Not Available],Complete Remission/Response


In [6]:
def get_clinical_data(files_list, root_dir):
    clinical_dfs = []
    for file in files_list:
        if ".txt" in file: 
            clinical_dfs.append(pd.read_csv(root_dir+'/'+file, sep = '\t', skiprows= 1))
    return clinical_dfs

In [7]:
clinical_dfs = get_clinical_data(files_list, root_dir)

In [8]:
#make all data lowercase
def lowercase_dataframe(clinical_dfs):
    lower_clinical_dfs = []
    for df in clinical_dfs:
        lower_clinical_dfs.append(df.applymap(lambda s:s.lower() if type(s) == str else s))
    return lower_clinical_dfs

In [9]:
clinical_dfs = lowercase_dataframe(clinical_dfs)

In [10]:
clinical_dfs[2]

,bcr_patient_uuid,bcr_patient_barcode,bcr_followup_barcode,bcr_followup_uuid,form_completion_date,followup_case_report_form_submission_reason,lost_follow_up,radiation_therapy,postoperative_rx_tx,primary_therapy_outcome_success,...,performance_status_scale_timing,new_tumor_event_after_initial_treatment,days_to_new_tumor_event_after_initial_treatment,additional_surgery_locoregional_procedure,days_to_additional_surgery_locoregional_procedure,additional_surgery_metastatic_procedure,days_to_additional_surgery_metastatic_procedure,additional_radiation_therapy,additional_pharmaceutical_therapy,followup_treatment_success
0,cde_id:,cde_id:2003301,cde_id:,cde_id:,cde_id:,cde_id:3233305,cde_id:61333,cde_id:2005312,cde_id:3397567,cde_id:2786727,...,cde_id:2792763,cde_id:3121376,cde_id:3392464,cde_id:3008755,cde_id:3408572,cde_id:3008757,cde_id:3408682,cde_id:3427615,cde_id:3427616,cde_id:3104050
1,73292c19-d6a8-4bc4-97bc-ccce54f264f8,tcga-2k-a9we,tcga-2k-a9we-f67095,b33c8002-5a5c-4000-8dfa-66b4a99f3585,2014-11-4,scheduled follow-up submission,no,no,no,complete remission/response,...,[not evaluated],no,[not applicable],[not available],[not applicable],[not available],[not applicable],[not available],[not available],complete remission/response
2,851a1157-e460-4794-8534-2eb6f0ae7468,tcga-2z-a9j1,tcga-2z-a9j1-f69179,192f85e9-a6aa-4bba-b5f9-0909fd8148c2,2014-12-30,scheduled follow-up submission,no,no,no,complete remission/response,...,post adjuvant therapy,no,[not applicable],[not available],[not applicable],[not available],[not applicable],[not available],[not available],complete remission/response
3,95cb399c-0a6f-4bd8-8146-36f367f30611,tcga-2z-a9j2,tcga-2z-a9j2-f69112,8f64dcac-44e4-4d14-a8ec-ce7729b5600d,2014-12-29,scheduled follow-up submission,yes,[unknown],[unknown],[unknown],...,[unknown],[unknown],[not available],[unknown],[not available],[unknown],[not available],[not available],[not available],[unknown]
4,da0289c6-ff84-4063-ba99-a0d0a338f6cd,tcga-2z-a9j5,tcga-2z-a9j5-f69180,44c82ae6-3b34-4780-b514-337d49b55dfd,2014-12-30,scheduled follow-up submission,no,no,no,[unknown],...,[not evaluated],no,[not applicable],[not available],[not applicable],[not available],[not applicable],[not available],[not available],[unknown]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
312,a5b3de4c-f3fa-414c-b809-59c3c1438aa7,tcga-y8-a8ry,tcga-y8-a8ry-f66629,bc82251f-1a8b-47a6-9fd2-dd734124e592,2014-10-23,scheduled follow-up submission,no,no,no,complete remission/response,...,[not available],no,[not applicable],[not available],[not applicable],[not available],[not applicable],[not available],[not available],complete remission/response
313,44598068-5220-4287-adad-0b4c3e21dc4b,tcga-y8-a8rz,tcga-y8-a8rz-f66777,bb27819e-fd14-4f0b-b3e8-e253e45f766c,2014-10-23,scheduled follow-up submission,no,no,no,complete remission/response,...,[not available],no,[not applicable],[not available],[not applicable],[not available],[not applicable],[not available],[not available],complete remission/response
314,b02a2698-d4d3-4977-a251-989694d60f73,tcga-y8-a8s0,tcga-y8-a8s0-f66799,3d3fb368-10eb-496d-b3d6-bc403c43fe67,2014-10-23,scheduled follow-up submission,no,no,no,complete remission/response,...,[not available],no,[not applicable],[not available],[not applicable],[not available],[not applicable],[not available],[not available],complete remission/response
315,b02a2698-d4d3-4977-a251-989694d60f73,tcga-y8-a8s0,tcga-y8-a8s0-f70757,b071c309-fd3b-49c5-9c78-2541cbfdd8ca,2015-9-2,scheduled follow-up submission,yes,[not available],[not available],[not available],...,[not available],[not available],[not available],[not available],[not available],[not available],[not available],[not available],[not available],[not available]


In [11]:
#set uiud to the index for all dfs
def uiud_index(lowercase_clinical_dfs):
    for df in lowercase_clinical_dfs:
        df.set_index(df.iloc[:,0], inplace = True)
    return lowercase_clinical_dfs

In [12]:
uiud_index(clinical_dfs)

[                                                          bcr_patient_uuid  \
 bcr_patient_uuid                                                             
 cde_id:                                                            cde_id:   
 8ee720b5-6f3a-4921-85ab-70e75c0835b0  8ee720b5-6f3a-4921-85ab-70e75c0835b0   
 8ee720b5-6f3a-4921-85ab-70e75c0835b0  8ee720b5-6f3a-4921-85ab-70e75c0835b0   
 3372a7d2-79cb-4cf0-9a7e-fe1a6b205daa  3372a7d2-79cb-4cf0-9a7e-fe1a6b205daa   
 3372a7d2-79cb-4cf0-9a7e-fe1a6b205daa  3372a7d2-79cb-4cf0-9a7e-fe1a6b205daa   
 ...                                                                    ...   
 ef2d7ef5-cdfc-48c9-b3fe-6eaf3be1106d  ef2d7ef5-cdfc-48c9-b3fe-6eaf3be1106d   
 3f0a2571-c088-4735-8152-61f6fd60ac63  3f0a2571-c088-4735-8152-61f6fd60ac63   
 28011111-4a01-4cdc-8d6b-7223fb2c501b  28011111-4a01-4cdc-8d6b-7223fb2c501b   
 28011111-4a01-4cdc-8d6b-7223fb2c501b  28011111-4a01-4cdc-8d6b-7223fb2c501b   
 28011111-4a01-4cdc-8d6b-7223fb2c501b  28011111-4a01

In [13]:
clinical_dfs[1].shape, clinical_dfs[2].shape

((704, 26), (317, 26))

In [14]:
#figure out the correct concat
clinical_df = pd.concat(clinical_dfs, axis = 0, join = 'outer', sort = True)
clinical_df.shape

(3328, 154)

In [15]:
clinical_df.columns

Index(['additional_pharmaceutical_therapy', 'additional_radiation_therapy',
       'additional_surgery_locoregional_procedure',
       'additional_surgery_metastatic_procedure',
       'additional_treatment_completion_success_outcome',
       'age_at_initial_pathologic_diagnosis', 'bcr_drug_barcode',
       'bcr_drug_uuid', 'bcr_followup_barcode', 'bcr_followup_uuid',
       ...
       'tumor_tissue_site_other', 'tumor_total_necrosis_percent', 'tumor_type',
       'tx_on_clinical_trial', 'vital_status', 'weight',
       'well_differentiated_liposarcoma_primary_dx', 'white_cell_count_result',
       'year_of_initial_pathologic_diagnosis',
       'year_of_tobacco_smoking_onset'],
      dtype='object', length=154)

In [16]:
pruned_cdf = clinical_df.drop(columns = ['additional_pharmaceutical_therapy','additional_radiation_therapy','additional_surgery_locoregional_procedure','additional_surgery_metastatic_procedure','bcr_drug_barcode','bcr_drug_uuid','bcr_followup_barcode','bcr_followup_uuid']) #drop useless rows

In [17]:
pruned_cdf= pruned_cdf.drop(columns = [
    'bcr_patient_barcode',
    'bcr_patient_uuid',
    'clinical_M',
    'clinical_N',
    'clinical_T'
])

In [18]:
pruned_cdf = pruned_cdf.drop(columns = [
    'clinical_trail_drug_classification',
    'contiguous_organ_invaded',
    'contiguous_organ_resection_site'
])

In [19]:
minimal_cdf = pruned_cdf.iloc[:,[0,1,2,6,18,24,26,29,35,40,44,45,60,88,89,92,96,116,122,123,132]]

In [20]:
minimal_cdf.drug_name[minimal_cdf.drug_name.notna()].value_counts().sum() #will probably either need to group these things or something else 

448

In [21]:
 pickle.dump(minimal_cdf, open( "minimal_cdf.pickle", "wb" ) )

##### so the vital status is located in a text file that contains all of the clinical information for all the patients in that project- file name for the first case = nationwidechildrens.org_clinical_patient_kirc.txt 
#drugs for each patient are located in another text file - file name for first case = nationwidechildrens.org_clinical_drug_kirc.txt
#the rna-seq file for this case is called - 	d7984046-2ea2-4b2e-ba78-2909ed46f649.FPKM.txt.gz

In [47]:
#next task- figure out how to the clinical info spreadsheet


In [48]:
#note for later from file search
# cases.case_id in ["set_id:AXD11pcSNya3Dok10xD9"] 
# and files.analysis.workflow_type in ["HTSeq - FPKM"]
# and files.data_format in ["txt"] and files.data_type 
# in ["Gene Expression Quantification"] and
# files.experimental_strategy in ["RNA-Seq"]